In [6]:
!pip install -q pyomo

In [7]:
from pyomo.environ import *
import numpy as np
import pandas as pd

In [8]:
#Reading Data
df = pd.read_csv("lab6_ex1.csv")

In [9]:
df

Warehouses  Hassan  Jodhpur  Trichy  Simla  Vijayawada  Dibrugarh  Raipur  \
0  Ahmedabad     827      817    1470   2272        1615       3143    1264   
1  Bengaluru     559     1832     751   3772         637       3773    1856   
2    Chennai    1109     2443     577   3865         798       3521    1984   
3      Delhi    2943     1471    4676   1337        2702       2741    2482   
4    Kolkata    3742     3089    2931   3199        2226       1739    1691   
5    Lucknow    2353     1160    3295    368        2369       2365    1293   
6     Mumbai    1356     1617    2045   2941        2334       4151    1937   
7  Hyderabad     195     1862    1331   2921         402       2825     563   
8     Demand     205      300     425    270         375        225     185   

   Agartala  AvailStock  
0      4465         350  
1      5459         185  
2      5303         305  
3      4148         180  
4      2128         205  
5      3642         175  
6      4287         325  
7      4441         370  
8       110        2095

In [10]:
df.columns

Index(['Warehouses', 'Hassan', 'Jodhpur', 'Trichy', 'Simla', 'Vijayawada',
       'Dibrugarh', 'Raipur', 'Agartala', 'AvailStock'],
      dtype='object')

## Question 1
Let C[w, m] be the cost of transporting one truck-load of EVs from w to m. Write
a general optimization problem to minimize the total transportation cost. Use appropriate
notations and define appropriate sets to be used in the optimization problem.


##Solution to Question 1

The objective Function:-

Since We need to minimize the cost of transportation so we are taking all the corresponding routes and assigning a varaible ($x_{i,j}$)  and multiply with their respective cost ($c_{i,j}$ values given in the csv file) and we will minimize this objective Function 

Minimize $\sum_{i=1,j=1}^{i=8,j=8} C_{i,j} X_{i,j} $

Constraints:
* Demand constraints: Sum of demands at each market need to satisfied so demand at each marked has to be equal to sum of the ev's transported from each warehouse to that location and So there are 8 constraints for each market.

$\sum_{i=1}^{i=8} X_{i,j} = c_{9,j}$  ∀ j ={1,2,3,4,5,6,7,8}

$c_{9,j}$ are the the last row values

* Availability constraints:Since each warehouse cannot ship out more than the quantity they have available. Sum of shipments from each warehouse to every marked has to be less than or equal to the avaliability of stock at the warehouse. So we will got 8 constraints for each warehouse

$\sum_{j=1}^{j=8} X_{i,j} <= c_{i,9}$  ∀ i ={1,2,3,4,5,6,7,8}

$c_{i,9}$ are the the last column values


In [11]:
model = ConcreteModel()

In [12]:
# No of warehouses
M = len(df.index)-1

In [13]:
# No of markets
N = len(df.columns)-2

In [14]:
row_indices = range(M)
col_indices = range(N)
print(row_indices)
print(col_indices)

range(0, 8)
range(0, 8)


In [15]:
#printing out all the costs
for i in row_indices:
  for j in col_indices:
    print('Cost[',i,'][',j+1,']:',df.iloc[i,j+1])

Cost[ 0 ][ 1 ]: 827
Cost[ 0 ][ 2 ]: 817
Cost[ 0 ][ 3 ]: 1470
Cost[ 0 ][ 4 ]: 2272
Cost[ 0 ][ 5 ]: 1615
Cost[ 0 ][ 6 ]: 3143
Cost[ 0 ][ 7 ]: 1264
Cost[ 0 ][ 8 ]: 4465
Cost[ 1 ][ 1 ]: 559
Cost[ 1 ][ 2 ]: 1832
Cost[ 1 ][ 3 ]: 751
Cost[ 1 ][ 4 ]: 3772
Cost[ 1 ][ 5 ]: 637
Cost[ 1 ][ 6 ]: 3773
Cost[ 1 ][ 7 ]: 1856
Cost[ 1 ][ 8 ]: 5459
Cost[ 2 ][ 1 ]: 1109
Cost[ 2 ][ 2 ]: 2443
Cost[ 2 ][ 3 ]: 577
Cost[ 2 ][ 4 ]: 3865
Cost[ 2 ][ 5 ]: 798
Cost[ 2 ][ 6 ]: 3521
Cost[ 2 ][ 7 ]: 1984
Cost[ 2 ][ 8 ]: 5303
Cost[ 3 ][ 1 ]: 2943
Cost[ 3 ][ 2 ]: 1471
Cost[ 3 ][ 3 ]: 4676
Cost[ 3 ][ 4 ]: 1337
Cost[ 3 ][ 5 ]: 2702
Cost[ 3 ][ 6 ]: 2741
Cost[ 3 ][ 7 ]: 2482
Cost[ 3 ][ 8 ]: 4148
Cost[ 4 ][ 1 ]: 3742
Cost[ 4 ][ 2 ]: 3089
Cost[ 4 ][ 3 ]: 2931
Cost[ 4 ][ 4 ]: 3199
Cost[ 4 ][ 5 ]: 2226
Cost[ 4 ][ 6 ]: 1739
Cost[ 4 ][ 7 ]: 1691
Cost[ 4 ][ 8 ]: 2128
Cost[ 5 ][ 1 ]: 2353
Cost[ 5 ][ 2 ]: 1160
Cost[ 5 ][ 3 ]: 3295
Cost[ 5 ][ 4 ]: 368
Cost[ 5 ][ 5 ]: 2369
Cost[ 5 ][ 6 ]: 2365
Cost[ 5 ][ 7 ]: 1293
Cost[ 5 ][ 8 ]: 3642


In [16]:
model.x = Var(row_indices,col_indices,domain = NonNegativeReals)

In [17]:
model.objective = Objective(expr = sum(df.iloc[i,j+1]*model.x[i,j] for i in row_indices for j in col_indices),sense = minimize)

In [18]:
model.constraints = ConstraintList()

In [19]:
#constraints 
for i in row_indices:
  model.constraints.add(expr = sum(model.x[i,j] for j in col_indices)<=df.iloc[i,N+1])
for j in col_indices:
  model.constraints.add(expr = sum(model.x[i,j] for i in row_indices)==df.iloc[M,j+1])

In [20]:
model.pprint()

4 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   16 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain              : Size : Members
        None :     2 : x_index_0*x_index_1 :   64 : {(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7)}
    x_index_0 : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
    

In [21]:
!apt-get install -y -qq coinor-cbc

Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 155569 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unpack .../4-coinor-libcbc3_2.9.9+repack1-1_amd64.deb ...
Unpacking coinor-libcbc3 (2.9.9+repack1-1) ...
Selecting p

In [22]:
opt_cbc = SolverFactory('cbc')

In [23]:
result = opt_cbc.solve(model)
print("Solver Status",result.solver.status)
print("Solver Termination condition",result.solver.termination_condition)

Solver Status ok
Solver Termination condition optimal


In [24]:
#display solution
print("\nObjective= ",model.objective())

print("\nDecision Variables")
for i in row_indices:
  for j in col_indices:
    if model.x[i,j].value != 0:
      print("Number of EV s transported from ",df['Warehouses'][i],'to',df.columns[j+1],':',model.x[i,j].value)
print("\nConstraints")
model.constraints.display()


Objective=  2192870.0

Decision Variables
Number of EV s transported from  Ahmedabad to Jodhpur : 300.0
Number of EV s transported from  Ahmedabad to Dibrugarh : 45.0
Number of EV s transported from  Ahmedabad to Raipur : 5.0
Number of EV s transported from  Bengaluru to Trichy : 120.0
Number of EV s transported from  Bengaluru to Vijayawada : 65.0
Number of EV s transported from  Chennai to Trichy : 305.0
Number of EV s transported from  Delhi to Simla : 95.0
Number of EV s transported from  Delhi to Dibrugarh : 85.0
Number of EV s transported from  Kolkata to Dibrugarh : 95.0
Number of EV s transported from  Kolkata to Agartala : 110.0
Number of EV s transported from  Lucknow to Simla : 175.0
Number of EV s transported from  Mumbai to Hassan : 205.0
Number of EV s transported from  Mumbai to Raipur : 120.0
Number of EV s transported from  Hyderabad to Vijayawada : 310.0
Number of EV s transported from  Hyderabad to Raipur : 60.0

Constraints
constraints : Size=16
    Key : Lower : B

##Question 7

Report the number of truck-loads of EVs that are transported (report only those values
that are nonzero) from warehouses to markets. Report the total cost ?

##Solution 7:
* Total cost = 2192870

***No of Ev's transported***
* Number of EV s transported from  Ahmedabad to Jodhpur : 300.0
* Number of EV s transported from  Ahmedabad to Dibrugarh : 45.0
* Number of EV s transported from  Ahmedabad to Raipur : 5.0
* Number of EV s transported from  Bengaluru to Trichy : 120.0
* Number of EV s transported from  Bengaluru to Vijayawada : 65.0
* Number of EV s transported from  Chennai to Trichy : 305.0
* Number of EV s transported from  Delhi to Simla : 95.0
* Number of EV s transported from  Delhi to Dibrugarh : 85.0
* Number of EV s transported from  Kolkata to Dibrugarh : 95.0
* Number of EV s transported from  Kolkata to Agartala : 110.0
* Number of EV s transported from  Lucknow to Simla : 175.0
* Number of EV s transported from  Mumbai to Hassan : 205.0
* Number of EV s transported from  Mumbai to Raipur : 120.0
* Number of EV s transported from  Hyderabad to Vijayawada : 310.0
* Number of EV s transported from  Hyderabad to Raipur : 60.0

***Question8***

Suppose the Lucknow-Simla link is disrupted and no transportation is possible on the
Lucknow-Simla links. Without changing the pyomo model, how will you solve this problem?
You are only allowed to change the .csv file.

***Solution8***

* Since it is a minimzation problem if you changed the cost of a route to some large value then it won't take that route in the optimization solution as it is minimization problem.So we wil gave 10000 cost for the route lucknow - simla
* As we need to change the model and the changing the cost only effects the objective function, so first change the value in the csv file and upload the new file and next we deactivate the original objective function and create the new objective function with the new cost data and solve using cbc

In [33]:
df1 = pd.read_csv("lab6_ex1_1.csv")

In [34]:
df1

Warehouses  Hassan  Jodhpur  Trichy  Simla  Vijayawada  Dibrugarh  Raipur  \
0  Ahmedabad     827      817    1470   2272        1615       3143    1264   
1  Bengaluru     559     1832     751   3772         637       3773    1856   
2    Chennai    1109     2443     577   3865         798       3521    1984   
3      Delhi    2943     1471    4676   1337        2702       2741    2482   
4    Kolkata    3742     3089    2931   3199        2226       1739    1691   
5    Lucknow    2353     1160    3295  10000        2369       2365    1293   
6     Mumbai    1356     1617    2045   2941        2334       4151    1937   
7  Hyderabad     195     1862    1331   2921         402       2825     563   
8     Demand     205      300     425    270         375        225     185   

   Agartala  AvailStock  
0      4465         350  
1      5459         185  
2      5303         305  
3      4148         180  
4      2128         205  
5      3642         175  
6      4287         325  
7      4441         370  
8       110        2095

In [35]:
#Solving using cbc
model.objective.deactivate()
model.objective = Objective(expr = sum(df1.iloc[i,j+1]*model.x[i,j] for i in row_indices for j in col_indices))
model.pprint()

This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


4 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   16 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain              : Size : Members
        None :     2 : x_index_0*x_index_1 :   64 : {(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7)}
    x_index_0 : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
    

In [37]:
result = opt_cbc.solve(model)
print("Solver Status",result.solver.status)
print("Solver Termination condition",result.solver.termination_condition)
#display solution
print("\nObjective= ",model.objective())

print("\nDecision Variables")
for i in row_indices:
  for j in col_indices:
    if model.x[i,j].value !=0 :
      print("Number of EV s transported from ",df['Warehouses'][i],'to',df.columns[j+1],':',model.x[i,j].value)
print("\nConstraints")
model.constraints.display()

Solver Status ok
Solver Termination condition optimal

Objective=  2380570.0

Decision Variables
Number of EV s transported from  Ahmedabad to Jodhpur : 300.0
Number of EV s transported from  Ahmedabad to Raipur : 50.0
Number of EV s transported from  Bengaluru to Trichy : 120.0
Number of EV s transported from  Bengaluru to Vijayawada : 65.0
Number of EV s transported from  Chennai to Trichy : 305.0
Number of EV s transported from  Delhi to Simla : 180.0
Number of EV s transported from  Kolkata to Dibrugarh : 95.0
Number of EV s transported from  Kolkata to Agartala : 110.0
Number of EV s transported from  Lucknow to Dibrugarh : 130.0
Number of EV s transported from  Lucknow to Raipur : 45.0
Number of EV s transported from  Mumbai to Hassan : 205.0
Number of EV s transported from  Mumbai to Simla : 90.0
Number of EV s transported from  Mumbai to Raipur : 30.0
Number of EV s transported from  Hyderabad to Vijayawada : 310.0
Number of EV s transported from  Hyderabad to Raipur : 60.0

Co

##Question 9
Report the new solution value and the nonzero flows in the network. Comment on your observations.


##Solution 9

the optimal cost value = 2380570.0

* Comments : Since we are doing the minimization problem and minimum cost (from lucknow to others) will be included if it possible and the minimum cost occured at lucknow - simla and now we are  removing that route by increasing cost. Since the minimum route is removed then it will take another route which has more cost then that so the optimal cost value was increased

No of EV's ::

* Number of EV s transported from  Ahmedabad to Jodhpur : 300.0
* Number of EV s transported from  Ahmedabad to Raipur : 50.0
* Number of EV s transported from  Bengaluru to Trichy : 120.0
* Number of EV s transported from  Bengaluru to Vijayawada : 65.0
* Number of EV s transported from  Chennai to Trichy : 305.0
* Number of EV s transported from  Delhi to Simla : 180.0
* Number of EV s transported from  Kolkata to Dibrugarh : 95.0
* Number of EV s transported from  Kolkata to Agartala : 110.0
* Number of EV s transported from  Lucknow to Dibrugarh : 130.0
* Number of EV s transported from  Lucknow to Raipur : 45.0
* Number of EV s transported from  Mumbai to Hassan : 205.0
* Number of EV s transported from  Mumbai to Simla : 90.0
* Number of EV s transported from  Mumbai to Raipur : 30.0
* Number of EV s transported from  Hyderabad to Vijayawada : 310.0
* Number of EV s transported from  Hyderabad to Raipur : 60.0


##Question10
Suppose in addition to Lucknow-Simla link, the Kolkata-Dibrugarh link and Delhi-Raipur
link are disrupted and no transportation is possible along these links as well. Make appropriate
changes only to the .csv file and solve the model.

##Solution 10 
Similar to what we did the above problem in order to remove given routes we will increase the cost of the route to some large value (we took 10000). So we changed the costs in the csv file and reload the data and since the cost will effects only object function we deactive the objective function and create the new objective function using the modified data

In [38]:
df2 = pd.read_csv("lab6_ex1_2.csv")
df2

Warehouses  Hassan  Jodhpur  Trichy  Simla  Vijayawada  Dibrugarh  Raipur  \
0  Ahmedabad     827      817    1470   2272        1615       3143    1264   
1  Bengaluru     559     1832     751   3772         637       3773    1856   
2    Chennai    1109     2443     577   3865         798       3521    1984   
3      Delhi    2943     1471    4676   1337        2702       2741   10000   
4    Kolkata    3742     3089    2931   3199        2226      10000    1691   
5    Lucknow    2353     1160    3295  10000        2369       2365    1293   
6     Mumbai    1356     1617    2045   2941        2334       4151    1937   
7  Hyderabad     195     1862    1331   2921         402       2825     563   
8     Demand     205      300     425    270         375        225     185   

   Agartala  AvailStock  
0      4465         350  
1      5459         185  
2      5303         305  
3      4148         180  
4      2128         205  
5      3642         175  
6      4287         325  
7      4441         370  
8       110        2095

In [39]:
#Solving using cbc
model.objective.deactivate()
model.objective = Objective(expr = sum(df2.iloc[i,j+1]*model.x[i,j] for i in row_indices for j in col_indices))
model.pprint()

This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


4 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   16 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain              : Size : Members
        None :     2 : x_index_0*x_index_1 :   64 : {(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7)}
    x_index_0 : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
    

In [40]:
result = opt_cbc.solve(model)
print("Solver Status",result.solver.status)
print("Solver Termination condition",result.solver.termination_condition)
#display solution
print("\nObjective= ",model.objective())

print("\nDecision Variables")
for i in row_indices:
  for j in col_indices:
    if model.x[i,j].value !=0:
      print("Number of EV s transported from ",df['Warehouses'][i],'to',df.columns[j+1],':',model.x[i,j].value)
print("\nConstraints")
model.constraints.display()

Solver Status ok
Solver Termination condition optimal

Objective=  2518200.0

Decision Variables
Number of EV s transported from  Ahmedabad to Jodhpur : 300.0
Number of EV s transported from  Ahmedabad to Dibrugarh : 50.0
Number of EV s transported from  Bengaluru to Trichy : 120.0
Number of EV s transported from  Bengaluru to Vijayawada : 65.0
Number of EV s transported from  Chennai to Trichy : 305.0
Number of EV s transported from  Delhi to Simla : 180.0
Number of EV s transported from  Kolkata to Raipur : 95.0
Number of EV s transported from  Kolkata to Agartala : 110.0
Number of EV s transported from  Lucknow to Dibrugarh : 175.0
Number of EV s transported from  Mumbai to Hassan : 205.0
Number of EV s transported from  Mumbai to Simla : 90.0
Number of EV s transported from  Mumbai to Raipur : 30.0
Number of EV s transported from  Hyderabad to Vijayawada : 310.0
Number of EV s transported from  Hyderabad to Raipur : 60.0

Constraints
constraints : Size=16
    Key : Lower : Body  : 

##Question 11
Report the new solution value and the nonzero flows in the network. Explain your
observations.

##Solution 11

* Optimal cost value = 2518200

* Observations :: Optimal cost value is increased from the previous problem.
Since we removed the two routes now in order to get optimial value the solver has taken the differnt routes which has higher cost then previous cost so the optimal value was increased

***No of Ev's transported***

* Number of EV s transported from  Ahmedabad to Jodhpur : 300.0
* Number of EV s transported from  Ahmedabad to Dibrugarh : 50.0
* Number of EV s transported from  Bengaluru to Trichy : 120.0
* Number of EV s transported from  Bengaluru to Vijayawada : 65.0
* Number of EV s transported from  Chennai to Trichy : 305.0
* Number of EV s transported from  Delhi to Simla : 180.0
* Number of EV s transported from  Kolkata to Raipur : 95.0
* Number of EV s transported from  Kolkata to Agartala : 110.0
* Number of EV s transported from  Lucknow to Dibrugarh : 175.0
* Number of EV s transported from  Mumbai to Hassan : 205.0
* Number of EV s transported from  Mumbai to Simla : 90.0
* Number of EV s transported from  Mumbai to Raipur : 30.0
* Number of EV s transported from  Hyderabad to Vijayawada : 310.0
* Number of EV s transported from  Hyderabad to Raipur : 60.0

